In [2]:
import numpy as np
import pickle
import os
import sys

In [23]:
src_folder = "E:/DevData/rehab_exercise/shoulder_surgery/data/"
with open(src_folder+'unprocessed.pkl', "rb") as file:
    data = pickle.load(file)

In [24]:
def resample(data, ts):        
    count = len(ts)    
    res = np.zeros((count, data.shape[1]))
    
    if data[0,0]>ts[0] or data[-1, 0]<ts[-1]:
        print(ts[0], ts[-1])
        print(data[0, 0], data[-1, 0])
        print("Accel time problem")
        sys.exit(0)
        
    i = 0
    j = 0
    while i < count:
        if ts[i] == data[j,0]:
            res[i, 0] = ts[i]
            res[i, 1:] = data[j, 1:]
            i+=1
        elif data[j, 0] < ts[i] < data[j+1, 0]:
            res[i, 0] = ts[i]        
            factor = (ts[i] - data[j, 0])/(data[j+1, 0]-data[j, 0]);
            res[i, 1:] = (1-factor)*data[j, 1:]  + factor*data[j+1, 1:]
            i+=1
        else:
            j+=1
            
    return res

def process_quat(q):
    #print(q.shape)
    #print(q[0, :])
    #q0 = q[:, 0]
    count = len(q)
    q1 = q[:, 0].reshape(count, 1)
    q2 = q[:, 1].reshape(count, 1)
    q3 = q[:, 2].reshape(count, 1)
    sin2_theta_by_2 = q1*q1 + q2*q2 + q3*q3        
    q0 = np.sqrt(1-sin2_theta_by_2)
        
    Rx = np.concatenate( ( q0*q0+q1*q1-q2*q2-q3*q3, 2*(q1*q2-q0*q3), 2*(q1*q3+q0*q2) ), axis=1)
    Ry = np.concatenate( ( 2*(q0*q3+q1*q2),  q0*q0-q1*q1+q2*q2-q3*q3, 2*(q2*q3 - q0*q1) ), axis=1)
    Rz = np.concatenate( ( 2*(q1*q3 - q0*q2), 2*(q2*q3+q0*q1), q0*q0-q1*q1-q2*q2+q3*q3 ), axis=1)
    
    R = np.concatenate((Rx, Ry, Rz), axis=1)
    
    return R

In [25]:
sensors = [1, 3, 4, 9, 10, 11]
print(len(data))
for i in range(8):
    #print(len(data[i][0]), len(data[i][1]), len(data[i][2]), data[i][2][0, -2], data[i][2][-1, -1])
    print("Subject ", i, ": ", data[i][2][0, -2])    
    
    d = data[i][0]
    a = d[d[:, 1]==1 , :]    
    g = d[d[:, 1]==4 , :]
    t1 = max(a[0, 0], g[0, 0])    
    t2 = min(a[-1, 0], g[-1, 0])    
    d = d[(d[:, 0]>=t1) & (d[:, 0]<=t2), :]
    
    accel = d[d[:, 1]==1, :]
    gyro = d[d[:, 1]==4, :] 
    grav = d[d[:, 1]==9, :]
    laccel = d[d[:, 1]==10, :]
    orient = d[d[:, 1]==3, :]
    quat = d[d[:, 1]==11, :]    
    accel = resample(accel[:, [0, 3, 4, 5]], grav[:, 0])
    gyro = resample(gyro[:, [0, 3, 4, 5]], grav[:, 0])    
    data[i][0] = np.concatenate((accel, gyro[:, 1:], grav[:, 3:], laccel[:, 3:], orient[:, 3:], process_quat(quat[:, 3:]) ), axis = 1)
    
    d = data[i][1]
    a = d[d[:, 1]==1 , :]    
    g = d[d[:, 1]==4 , :]
    t1 = max(a[0, 0], g[0, 0])    
    t2 = min(a[-1, 0], g[-1, 0])    
    d = d[(d[:, 0]>=t1) & (d[:, 0]<=t2), :]
    
    accel = d[d[:, 1]==1, :]
    gyro = d[d[:, 1]==4, :]    
    grav = d[d[:, 1]==9, :]
    laccel = d[d[:, 1]==10, :]
    orient = d[d[:, 1]==3, :]
    quat = d[d[:, 1]==11, :]    
    accel = resample(accel[:, [0, 3, 4, 5]], grav[:, 0])
    gyro = resample(gyro[:, [0, 3, 4, 5]], grav[:, 0])
    data[i][1] = np.concatenate((accel, gyro[:, 1:], grav[:, 3:], laccel[:, 3:], orient[:, 3:], process_quat(quat[:, 3:]) ), axis = 1)

8
Subject  0 :  220.8
Subject  1 :  2200.0
Subject  2 :  228.1
Subject  3 :  164.8
Subject  4 :  1810.8
Subject  5 :  3130.7
Subject  6 :  250.6
Subject  7 :  1699.6


In [26]:
with open(src_folder+'processed.pkl', "wb") as file:
    pickle.dump(data, file)

In [27]:
for i in range(8):
    print("Subject ", i, ": ", data[i][0].shape, data[i][1].shape, data[i][2].shape)
    print(">>>  ", data[i][0][0, 0], data[i][0][-1, 0])
    print(">>>  ", data[i][1][0, 0], data[i][1][-1, 0])
    print(">>>  ", data[i][2][0, -2], data[i][2][-1, -1])

Subject  0 :  (100541, 25) (98707, 25) (61, 5)
>>>   53.1585645335 2059.99936846
>>>   89.820555052 2059.99319638
>>>   220.8 2047.5
Subject  1 :  (81292, 25) (81681, 25) (60, 5)
>>>   2170.00484003 3792.64049493
>>>   2170.01404462 3800.41062036
>>>   2200.0 3744.4
Subject  2 :  (90818, 25) (86957, 25) (66, 5)
>>>   56.4947642276 1869.2236464
>>>   118.181025833 1853.8433059
>>>   228.1 1801.6
Subject  3 :  (85314, 25) (83968, 25) (63, 5)
>>>   37.0358400225 1739.99203495
>>>   63.98419724 1739.98838671
>>>   164.8 1712.6
Subject  4 :  (61621, 25) (61621, 25) (66, 5)
>>>   1770.00880952 2999.9860165
>>>   1770.00922372 2999.98348435
>>>   1810.8 2956.0
Subject  5 :  (90346, 25) (89846, 25) (67, 5)
>>>   3030.01372503 4833.42484476
>>>   3030.01374179 4823.45168376
>>>   3130.7 4781.0
Subject  6 :  (71571, 25) (72060, 25) (71, 5)
>>>   131.404896459 1559.99417638
>>>   121.600497136 1559.98382096
>>>   250.6 1524.8
Subject  7 :  (63124, 25) (63123, 25) (48, 5)
>>>   1620.01242037 2879.

In [28]:
print(data[0][0][:10, :])

[[  5.31585645e+01  -6.70376500e-02   2.39899020e+00   9.65581500e+00
   -2.40217130e-01   4.24507900e-01   2.50070840e-01  -8.81260200e+00
    3.30814960e+00   2.75035100e+00   8.74556400e+00  -9.09159400e-01
    6.90546400e+00   2.15178710e+02  -1.97147350e+01  -7.26672200e+01
   -4.32315634e-01  -5.72362663e-01  -6.96781296e-01  -7.45243981e-02
   -7.47400050e-01   6.60181248e-01  -8.98637472e-01   3.37333882e-01
    2.80457744e-01]
 [  5.31766812e+01  -1.67594120e-01   2.33913520e+00   9.90241800e+00
   -2.21308700e-01   4.07996300e-01   2.60723470e-01  -8.76904300e+00
    3.33441100e+00   2.85583190e+00   8.60144900e+00  -9.95275740e-01
    7.04658600e+00   2.09777200e+02  -1.98778060e+01  -7.19610100e+01
   -4.33993439e-01  -4.98807873e-01  -7.50226899e-01  -1.09824255e-01
   -7.97235539e-01   5.93594245e-01  -8.94197030e-01   3.40009118e-01
    2.91213790e-01]
 [  5.32004598e+01  -4.54898330e-01   2.03507140e+00   9.64863300e+00
   -1.54995980e-01   3.83228900e-01   1.81361270e-